In [18]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from Models.MMC import MMC, sgo_types
import datetime
import random
from numpy.random import choice
from markov import MarkovChain
from Models.MMC import MMC, sgo_types
from Models.CAP import CAP
from Models.HMC import HMC
from Models.FMC import FMC

from Models.MAX import MAX

from Datasets import Markov_Data, MMC_Data, Fruit_Data

from mtdlearn.mtd import MTD
from mtdlearn.datasets import ChainGenerator

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Generate data
First, generate a standard 1st order Markov Model cg.

In [ ]:
states = ('A', 'B', 'C', 'D')
order = 3
dataset_size = 2000
cg = ChainGenerator(states, order=1, min_len= 3, max_len= 3)

In [142]:
cg.transition_matrix

array([[0.34532058, 0.37017921, 0.24721113, 0.03728908],
       [0.0302618 , 0.42666673, 0.13088301, 0.41218846],
       [0.1370596 , 0.17778852, 0.28516126, 0.39999062],
       [0.40679655, 0.10643655, 0.13675736, 0.35000954]])

In [5]:
MMC_Data.MMC_data.gen_data(4,3,20000)

[array([[3, 0, 0],
        [3, 0, 3],
        [3, 1, 1],
        ...,
        [2, 3, 2],
        [1, 2, 2],
        [3, 1, 1]]),
 array([[2, 0, 3],
        [1, 0, 1],
        [3, 1, 3],
        ...,
        [2, 1, 1],
        [0, 1, 2],
        [0, 0, 2]]),
 array([1, 1, 1, ..., 0, 2, 0]),
 array([2, 3, 1, ..., 0, 2, 1])]

## Randomly Select One SGO
Now from the set of all SGOs randomly select one SGO

In [133]:
labels = {v: k for k, v in cg._label_dict.items()}
state_num = list(labels.values())
SGO = random.sample(state_num, len(state_num))
x = [random.choices(state_num, k=order) for i in range(dataset_size)]
y = [choice(state_num, 1, p=cg.transition_matrix[MMC.find_high(x_, SGO)])[0] for x_ in x ]

In [117]:
SGO

[2, 3, 1, 0]

In [118]:
labels.values()

dict_values([0, 1, 2, 3])

## Generate input and output data
First we will generate 100,000 random sequences of order 3 states. Then we will select the highest order state from each sequence and sample from it's associated entry in the markov table.

For example, if the episode is ['C', 'B', 'C'] and the SGO ['C', 'A', 'D', 'B'], the 'C' distribution will be used to generate the output.

In [28]:
state_count = 4
X_train, X_test, y_train, y_test = MMC_Data.MMC_data.gen_data(state_count, 3, 20000, True)## Fitting model

[[0.1899279  0.4559948  0.04866433 0.30541297]
 [0.24372757 0.50495051 0.03226755 0.21905437]
 [0.01566661 0.09741709 0.13090704 0.75600927]
 [0.13590479 0.43521085 0.3003318  0.12855257]]
[2, 1, 0, 3]


In [90]:

arguments = {"X_train":X_train,"y_train":y_train,"states":labels}

arguments["sgo_method"] = sgo_types.hillclimb
model0 = MMC(**arguments)

arguments["sgo_method"] = sgo_types.greedy
model1 = MMC(arguments)

arguments["sgo_method"] = sgo_types.full
model2 = MMC(arguments)


TypeError: MMC.__init__() got an unexpected keyword argument 'X_train'

## Convert Probabilities
Now we will convert the dictionary format into a traditional CPT table format

Now we must compare these two arrays to verify training is done accurately

In [ ]:
state_count = 4

amount_to_average = 5

training_master = []
testing_master = []
# implement FMC
o = 3
methods = [MMC, MTD, CAP, HMC, MAX, FMC]
types = ["MMC", "MTD", "CAP", "HMC", "MAX", "FMC"]
for _ in range(amount_to_average):

    X_train, X_test, y_train, y_test = MMC_Data.MMC_data.gen_data(state_count,3,30000, True)## Fitting model
    args_training = {"X_train": X_train, "y_train": y_train}
    args_testing = {"X_test": X_test, "y_test": y_test}
    results_training = []
    results_testing = []

    for m in methods:
        model = m(state_count, order=o)
        results_training.append(MarkovChain.calculate_time(model.train, args_training))
        results_testing.append(MarkovChain.calculate_time(model.test, args_testing))

    training_master.append(results_training)
    testing_master.append(results_testing)

KeyboardInterrupt: 

In [14]:
training_master

[[(0.3549333333333333, 5.521003),
  (0.34986666666666666, 0.819414),
  (0.33413333333333334, 3.610424),
  (0.38866666666666666, 1.061475),
  (0.33413333333333334, 3.642435),
  (0.34026666666666666, 1.073637)],
 [(0.504, 5.35482),
  (0.45613333333333334, 0.064922),
  (0.4472, 3.609104),
  (0.5257333333333334, 1.07793),
  (0.4472, 3.591545),
  (0.4472, 1.066975)],
 [(0.32266666666666666, 5.376923),
  (0.3352, 0.074048),
  (0.3104, 3.61515),
  (0.37906666666666666, 1.090463),
  (0.3104, 3.582452),
  (0.3156, 1.065514)],
 [(0.3672, 5.402237),
  (0.31733333333333336, 0.05416),
  (0.2876, 3.582243),
  (0.38666666666666666, 1.057701),
  (0.2996, 3.58449),
  (0.31266666666666665, 1.085489)],
 [(0.37346666666666667, 5.431063),
  (0.3516, 0.048201),
  (0.3516, 3.565828),
  (0.38693333333333335, 1.070125),
  (0.3516, 3.506565),
  (0.3516, 1.056651)]]

In [23]:
def find_average(arr):
    return sum(arr)/len(arr)

In [ ]:
acc_testing = []
for i in range(len(methods)):
    acc_testing += [find_average([av_[i][0] for av_ in training_master])]

acc_training = []
for i in range(len(methods)):
    acc_training += [find_average([av_[i][0] for av_ in testing_master])]


training_times = []
for i in range(len(methods)):
    training_times += [find_average([av_[i][1] for av_ in training_master])]

In [25]:
import numpy as np
import matplotlib.pyplot as plt


# creating the dataset
def create_bar_graph(data, title):
    courses = list(data.keys())
    values = list(data.values())

    plt.figure(figsize = (10, 5))

    # creating the bar plot
    plt.bar(courses, values, color ='maroon',
            width = 0.4)

    plt.title(title)
    plt.show()

In [27]:
acc_training

[0.43,
 0.40836000000000006,
 0.39692,
 0.43431999999999993,
 0.39692,
 0.39855999999999997]

In [ ]:

create_bar_graph({types[i] : acc_training[i] for i in range(len(acc_training))}, "Training Accuracy")
create_bar_graph({types[i] : acc_testing[i] for i in range(len(acc_testing))}, "Testing Accuracy")
create_bar_graph({types[i] : training_times[i] for i in range(len(training_times))}, "Training Times")

In [127]:
#sum([ 1 for i, lag in enumerate(X_test) if np.argmax(model2.cpt[MAX.find_high(lag, SGO)]) == y_test[i]])/len(y_test)

def test_model(model):
    return sum([ 1 for i, lag in enumerate(X_test) if np.argmax(model.cpt[MAX.find_high(lag, model.SGO)]) == y_test[i]])/len(y_test)

test_model(model2)

0.42112

In [ ]:
# MMC is the new one
# implement CAP, HMC, MTD, FMC, MAX SMILE, MAX COUNTING
from markov import MarkovChain
o = 3
methods = [CAP, MMC, HMC, MAX]
results = []
training_times = []
args_training = {"X_train": X_train, "y_train": y_train}
args_testing = {"X_test": X_test, "y_test": y_test}
for m in methods:
    model = m(labels, o)
    training_times.append(MarkovChain.calculate_time(model.train, args_training))
    results.append(MarkovChain.calculate_time(model.test, args_testing))


In [74]:
models = []
for i in range(1, 4):
    models.append(MTD(order=i))

In [75]:
save_path = "/home/mitch/PycharmProjects/diverse-plan-rec/src/pr/input_data/generated_markov_data/"

for gap in range(0,X_train.shape[1]):
    col = X_train.shape[1]-gap-1
    print(X_train[:,col])
    pd.DataFrame({'state0': X_train[:,col], 'state1':y_train}).to_csv(save_path+f"train_data_gap{gap+1}_train.txt", index=False)

[3 2 3 ... 3 3 3]
[2 2 3 ... 2 0 3]
[2 3 0 ... 3 0 0]


In [76]:
import pickle
with open(f"{save_path}/test_data.pickle", 'wb') as handle:
    pickle.dump([X_test, y_test], handle)

In [77]:
same_input = [i for i, row in enumerate(X_train == X_train[0]) if row.all()]
[y_train[s] for s in same_input]
# make sure all the values are the same ie they are deterministic

[3,
 3,
 2,
 2,
 0,
 3,
 2,
 0,
 2,
 0,
 3,
 3,
 0,
 0,
 3,
 2,
 0,
 3,
 3,
 0,
 0,
 3,
 2,
 3,
 3,
 0,
 0,
 2,
 3,
 2,
 2,
 3,
 0,
 3,
 3,
 3,
 2,
 3,
 2,
 3,
 2,
 0,
 3,
 1,
 3,
 3,
 2,
 0,
 3,
 0,
 1,
 3,
 3,
 2,
 0,
 2,
 2,
 0,
 3,
 0,
 3,
 2,
 2,
 0,
 0,
 2,
 2,
 3,
 1,
 1,
 3,
 2,
 0,
 3,
 1,
 3,
 0,
 0,
 3,
 2,
 0,
 2,
 2,
 3,
 2,
 0,
 1,
 2,
 2,
 3,
 0,
 3,
 0,
 2,
 3,
 3,
 3,
 3,
 3,
 2,
 1,
 0,
 2,
 3,
 3,
 2,
 3,
 2,
 0,
 3,
 3,
 3,
 3,
 0,
 0,
 2,
 0,
 2,
 0,
 1,
 1,
 0,
 2,
 2,
 0,
 2,
 0,
 3,
 3,
 2,
 2,
 3,
 1,
 3,
 2,
 1,
 0,
 3,
 0,
 0,
 3,
 3,
 0,
 0,
 3,
 3,
 3,
 0,
 0,
 3,
 3,
 0,
 3,
 0,
 2,
 0,
 0,
 3,
 0,
 3,
 3,
 3,
 0,
 0,
 2,
 3,
 0,
 0,
 3,
 0,
 3,
 0,
 3,
 3,
 3,
 3,
 3,
 3,
 2,
 2,
 2,
 1,
 3,
 0,
 2,
 3,
 3,
 3,
 2,
 2,
 3,
 1,
 1,
 2,
 1,
 3,
 2,
 0,
 0,
 0,
 2,
 3,
 3,
 1,
 0,
 1,
 2,
 3,
 0,
 2,
 2,
 3,
 1,
 2,
 0,
 3,
 2,
 3,
 1,
 2,
 2,
 3,
 3,
 2,
 3,
 0,
 3,
 2,
 3,
 3,
 3,
 3,
 2,
 1,
 2,
 0,
 3,
 3,
 3,
 0,
 3,
 2,
 3,
 0,
 1,
 3,
 2,
 1,
 2,
 2,


In [78]:
def save_data(x,y, name):
    values = [np.concatenate((x[i],[y[i]], ["end"])) for i in range(len(x))]
    train_markov_data = pd.DataFrame({'state': np.append([],values)})
    train_markov_data.to_csv(save_path+name, index=False)
save_data(X_train, y_train, "train_markov.txt")
save_data(X_test,  y_test, "test.txt")


In [79]:
from datetime import datetime

print("Training Times:")

for i, model in enumerate(models):
    start = datetime.now()
    model.fit(X_train[:,len(models)-i-1:], y_train)
    print(datetime.now() - start)

Training Times:
log-likelihood value: -411799.462097521
0:00:00.979866
log-likelihood value: -407206.93467911996
0:00:00.224420
log-likelihood value: -402889.63620540156
0:00:00.351516


In [80]:
X_train[:,len(models)-i-1:]

array([[2, 2, 3],
       [3, 2, 2],
       [0, 3, 3],
       ...,
       [3, 2, 3],
       [0, 0, 3],
       [0, 3, 3]])

## Information criteria

In [81]:
model.aic

805839.2724108031

In [82]:
model.bic

806157.6185434123

## Trained parameters

In [83]:
model.lambdas.round(3)

array([0.577, 0.307, 0.116])

In [84]:
model.transition_matrices.round(3)

array([[[0.245, 0.251, 0.274, 0.23 ],
        [0.135, 0.205, 0.307, 0.353],
        [0.328, 0.148, 0.191, 0.333],
        [0.172, 0.211, 0.126, 0.491]],

       [[0.36 , 0.281, 0.032, 0.326],
        [0.182, 0.523, 0.071, 0.224],
        [0.13 , 0.203, 0.313, 0.354],
        [0.27 , 0.423, 0.269, 0.038]],

       [[0.019, 0.115, 0.39 , 0.477],
        [0.369, 0.087, 0.404, 0.139],
        [0.283, 0.452, 0.179, 0.086],
        [0.121, 0.023, 0.367, 0.488]]])

## Predict

In [87]:
result = []
for i, model in enumerate(models):
    result.append(len([y for y,p in enumerate(model.predict(X_test[:,len(models)-i-1:4])) if p == y_test[y]])/len(y_test))

In [88]:
result

[0.30926, 0.32556, 0.33925]

In [89]:
sum(list(map(np.argmax,cg.predict_proba(np.array(X_test)))) == y_test)/len(y_test)

0.34199

In [90]:
import numpy as np
from sklearn.preprocessing import normalize

In [91]:

from itertools import permutations

gap_probs = [{},{},{}]

for i in range(len(X_test[0])):
    for s in map(states.index, states):
        not_s = [states.index(x) for x in states if states.index(x) != s ]
        probs = []
        for p in permutations(not_s):
            input = [-1 for _ in range(cg.order)]
            input[i] = s

            c = 0
            for r in set([j for j in range(cg.order) if j != i]):
                input[r] = p[c]
                c += 1

            probs.append(cg.predict_proba(np.array([input])))
        sum_list = [sum([probs[j][i] for j in range(len(probs))]) for i in range(len(probs[0]))]
        gap_probs[i][s] =  [s/sum(sum_list[0]) for s in sum_list[0]]

In [60]:
gap_probs
# gap 1
# gap 2
# gap 3
# Array is in that orderc

[{0: [0.3409188300864065,
   0.29629403735781173,
   0.17716289656421638,
   0.1856242359915654],
  1: [0.403694423663215,
   0.20823007401341442,
   0.2552413594309092,
   0.13283414289246134],
  2: [0.27904897032076886,
   0.3330698321785714,
   0.09373260722424294,
   0.2941485902764169],
  3: [0.37818337835859756,
   0.21208721769174796,
   0.16956201107920463,
   0.24016739287044983]},
 {0: [0.4210396168583954,
   0.12243077675564094,
   0.1552396179326447,
   0.30128998845331895],
  1: [0.26364796816813635,
   0.28017823324079694,
   0.13007624654086827,
   0.32609755205019847],
  2: [0.3859215441701222,
   0.2890435036854268,
   0.20913388498321714,
   0.11590106716123394],
  3: [0.33123647323233396,
   0.35802864755968083,
   0.201249124841843,
   0.1094857543661421]},
 {0: [0.30094046749374576,
   0.31376862748704226,
   0.18897475319647977,
   0.19631615182273215],
  1: [0.36301437117240737,
   0.28400034560314086,
   0.15862325252161624,
   0.1943620307028354],
  2: [0.38305

In [61]:
from itertools import chain as ch
sum(list(map(lambda x: np.argmax(x) % len(states), [list(ch(*[gap_probs[gap-1][item[-gap]] for gap in range(len(gap_probs))])) for item in X_test])) == y_test) / len(y_test)

0.35325

In [62]:
np.argmax(list(ch(*[gap_probs[gap-1][item[-gap]] for gap in range(len(gap_probs))])))

NameError: name 'item' is not defined

In [ ]:
[gap_probs[gap-1][item[-gap]] for gap in range(len(gap_probs))]

In [ ]:
list(ch(*[gap_probs[gap-1][item[-gap]] for gap in range(len(gap_probs))]))

In [ ]:
sum(list(map(np.argmax,cg.predict_proba(np.array(X_test)))) == y_test)/len(y_test)

In [ ]:
# first is
gap_probs[0][0]

In [ ]:
cg._label_dict.values

In [ ]:
cg._label_dict

In [ ]:
np.array(X_test)